In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

experiment_selector = widgets.ExperimentSelectionWidget()
experiment_selector.load_dropdown_box()

misc_widgets = widgets.ObjectPropertiesWidgets("Miscellaneous")
misc_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def upload_datasets(method_object, support_files_widget):
    for filename in support_files_widget.value:
        file_info = support_files_widget.value[filename]
        utils.save_file(file_info['content'], filename)
        OPENBIS_SESSION.new_dataset(type = 'RAW_DATA', sample = method_object, files = [filename]).save()
        os.remove(filename)

def create_miscellaneous_action(b):
    if experiment_selector.dropdown.value == -1:
        print("Select an experiment.")
        return

    object_properties = {
        "$name": misc_widgets.properties_widgets["$name"].value, 
        "comments": misc_widgets.properties_widgets["comments"].value
    }
    
    method_object = utils.create_openbis_object(
        OPENBIS_SESSION, 
        type = CONFIG["objects"]["Miscellaneous"]["openbis_object_type"],
        collection = experiment_selector.dropdown.value, props = object_properties
    )
    upload_datasets(method_object, misc_support_files)
    print("Upload successful!")

# Others

## Select experiment

In [ ]:
display(experiment_selector)

display(utils.Markdown(data = "### Properties"))
display(misc_widgets)

display(utils.Markdown(data = "### Support Files"))
display(misc_support_files)

display(utils.Markdown(data = "### Save"))
display(save_close_buttons_hbox)
display(increase_buttons_size)

create_button.on_click(create_miscellaneous_action)
quit_button.on_click(close_notebook)
